<a href="https://colab.research.google.com/github/nvnanava/Yelp_Project/blob/main/Assignment1_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import needed packages

In [21]:
!pip install pandas
!pip install scikit-learn
!pip install tensorflow

Import needed packages part 2

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_squared_error
import random
from typing import List, Tuple, Dict
import warnings
warnings.filterwarnings('ignore')


Import reviews and business data

Note: Reading business file, then reducing it to those businesses that have more than 20 reviews.

In [23]:
#reviews_df = pd.read_json("yelp_academic_dataset_review.json", lines=True)
temp_df = pd.read_json("Yelp JSON/yelp_dataset/yelp_academic_dataset_business.json", lines=True)
business_df = temp_df[temp_df['review_count'] >= 20]
business_df


,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
2,tUFrWirKiKi_TAnsVWINQQ,Target,5255 E Broadway Blvd,Tucson,AZ,85711,32.223236,-110.880452,3.5,22,0,"{'BikeParking': 'True', 'BusinessAcceptsCredit...","Department Stores, Shopping, Fashion, Home & G...","{'Monday': '8:0-22:0', 'Tuesday': '8:0-22:0', ..."
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ..."
12,il_Ro8jwPlHresjw9EGmBg,Denny's,8901 US 31 S,Indianapolis,IN,46227,39.637133,-86.127217,2.5,28,1,"{'RestaurantsReservations': 'False', 'Restaura...","American (Traditional), Restaurants, Diners, B...","{'Monday': '6:0-22:0', 'Tuesday': '6:0-22:0', ..."
14,0bPLkL0QhhPO5kt1_EXmNQ,Zio's Italian Market,2575 E Bay Dr,Largo,FL,33771,27.916116,-82.760461,4.5,100,0,"{'OutdoorSeating': 'False', 'RestaurantsGoodFo...","Food, Delis, Italian, Bakeries, Restaurants","{'Monday': '10:0-18:0', 'Tuesday': '10:0-20:0'..."
15,MUTTqe8uqyMdBl186RmNeA,Tuna Bar,205 Race St,Philadelphia,PA,19106,39.953949,-75.143226,4.0,245,1,"{'RestaurantsReservations': 'True', 'Restauran...","Sushi Bars, Restaurants, Japanese","{'Tuesday': '13:30-22:0', 'Wednesday': '13:30-..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150330,GeEveoOaU2YKD7jJtEfA_g,DeVons Jewelers,"13981 S Virginia St, 402A",Reno,NV,89511,39.404552,-119.750986,5.0,34,1,"{'BusinessParking': '{'garage': False, 'street...","Shopping, Watches, Jewelry","{'Monday': '10:0-19:0', 'Tuesday': '10:0-19:0'..."
150331,qQ7FHvkGEMqoPKKXPk4gjA,La Quinta by Wyndham NW Tucson Marana,6020 West Hospitality Rd,Tucson,AZ,85743,32.358587,-111.093308,2.5,67,1,"{'RestaurantsPriceRange2': '2', 'BusinessAccep...","Hotels & Travel, Hotels, Event Planning & Serv...","{'Monday': '0:0-0:0', 'Tuesday': '0:0-0:0', 'W..."
150334,LJ4GjQ1HL6kqvIPpNUNNaQ,Shanti Yoga and Ayurveda,"1638 Pine St, Fl 1",Philadelphia,PA,19103,39.945966,-75.169666,4.5,39,1,"{'ByAppointmentOnly': 'True', 'GoodForKids': '...","Health & Medical, Yoga, Shopping, Naturopathic...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ..."
150336,WnT9NIzQgLlILjPT0kEcsQ,Adelita Taqueria & Restaurant,1108 S 9th St,Philadelphia,PA,19147,39.935982,-75.158665,4.5,35,1,"{'WheelchairAccessible': 'False', 'Restaurants...","Restaurants, Mexican","{'Monday': '11:0-22:0', 'Tuesday': '11:0-22:0'..."


In [24]:
# Saving using pickle for faster load later
business_df.to_pickle("businessdf.pkl")

In [25]:
# To load in the future....
business_df = pd.read_pickle("businessdf.pkl")

In [26]:
restaurant_df= business_df[business_df["categories"].str.contains("Restaurants", na = False)]

In [27]:
gym_df = business_df[business_df["categories"].str.contains("Fitness", na = False)]

In [28]:
doctor_df = business_df[business_df["categories"].str.contains("Health & Medical", na = False)]

In [29]:
jewel_df = business_df[business_df["categories"].str.contains("Jewelry", na = False)]

In [30]:
hotel_df = business_df[business_df["categories"].str.contains("Hotels", na = False)]

In [31]:
restaurant_df

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ..."
12,il_Ro8jwPlHresjw9EGmBg,Denny's,8901 US 31 S,Indianapolis,IN,46227,39.637133,-86.127217,2.5,28,1,"{'RestaurantsReservations': 'False', 'Restaura...","American (Traditional), Restaurants, Diners, B...","{'Monday': '6:0-22:0', 'Tuesday': '6:0-22:0', ..."
14,0bPLkL0QhhPO5kt1_EXmNQ,Zio's Italian Market,2575 E Bay Dr,Largo,FL,33771,27.916116,-82.760461,4.5,100,0,"{'OutdoorSeating': 'False', 'RestaurantsGoodFo...","Food, Delis, Italian, Bakeries, Restaurants","{'Monday': '10:0-18:0', 'Tuesday': '10:0-20:0'..."
15,MUTTqe8uqyMdBl186RmNeA,Tuna Bar,205 Race St,Philadelphia,PA,19106,39.953949,-75.143226,4.0,245,1,"{'RestaurantsReservations': 'True', 'Restauran...","Sushi Bars, Restaurants, Japanese","{'Tuesday': '13:30-22:0', 'Wednesday': '13:30-..."
19,ROeacJQwBeh05Rqg7F6TCg,BAP,1224 South St,Philadelphia,PA,19147,39.943223,-75.162568,4.5,205,1,"{'NoiseLevel': 'u'quiet'', 'GoodForMeal': '{'d...","Korean, Restaurants","{'Monday': '11:30-20:30', 'Tuesday': '11:30-20..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150319,8n93L-ilMAsvwUatarykSg,Kitchen Gia,3716 Spruce St,Philadelphia,PA,19104,39.951018,-75.198240,3.0,22,0,"{'RestaurantsGoodForGroups': 'True', 'BikePark...","Coffee & Tea, Food, Sandwiches, American (Trad...","{'Monday': '9:0-19:30', 'Tuesday': '9:0-19:30'..."
150322,2MAQeAqmD8enCT2ZYqUgIQ,The Melting Pot - Nashville,"166 2nd Ave N, Ste A",Nashville,TN,37201,36.163875,-86.776311,4.0,204,0,"{'RestaurantsDelivery': 'False', 'RestaurantsR...","Fondue, Beer, Wine & Spirits, Food, Restaurants","{'Monday': '0:0-0:0', 'Tuesday': '16:0-21:0', ..."
150323,w_4xUt-1AyY2ZwKtnjW0Xg,Bittercreek Alehouse,246 N 8th St,Boise,ID,83702,43.616590,-116.202383,4.5,998,1,"{'BikeParking': 'True', 'Alcohol': 'u'full_bar...","Bars, Gastropubs, Sandwiches, Nightlife, Resta...","{'Monday': '0:0-0:0', 'Tuesday': '11:0-22:0', ..."
150327,cM6V90ExQD6KMSU3rRB5ZA,Dutch Bros Coffee,1181 N Milwaukee St,Boise,ID,83704,43.615401,-116.284689,4.0,33,1,"{'WiFi': ''free'', 'RestaurantsGoodForGroups':...","Cafes, Juice Bars & Smoothies, Coffee & Tea, R...","{'Monday': '0:0-0:0', 'Tuesday': '0:0-17:0', '..."


In [32]:
gym_df

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
261,I3ZLcrGzj-QN5vFgaJFrAg,Golf Galaxy,470 Swedesford Rd,Berwyn,PA,19312,40.067670,-75.449697,3.0,25,1,"{'GoodForKids': 'True', 'BusinessAcceptsCredit...","Shopping, Fashion, Outdoor Gear, Active Life, ...","{'Monday': '10:0-20:0', 'Tuesday': '10:0-20:0'..."
262,Tm3QiAVLJAR2fEJK7APPPQ,City Fitness - Northern Liberties,200 Spring Garden St,Philadelphia,PA,19123,39.960343,-75.141951,3.0,131,1,"{'BusinessAcceptsCreditCards': 'True', 'GoodFo...","Active Life, Fitness & Instruction, Trainers, ...","{'Monday': '0:0-0:0', 'Tuesday': '5:0-1:0', 'W..."
397,HsibLfc2XD7Gh7Hbd1WfJQ,Anytime Fitness,"110 W Main St, Ste 170",Carmel,IN,46032,39.979147,-86.128575,2.5,22,1,"{'BusinessParking': '{'garage': False, 'street...","Active Life, Gyms, Fitness & Instruction, Trai...","{'Monday': '0:0-0:0', 'Tuesday': '0:0-0:0', 'W..."
528,kxvOUehCtJAL2nYKkeiR-w,Incrediplex,6002 Sunnyside Rd,Indianapolis,IN,46236,39.866083,-85.975098,4.0,33,0,"{'GoodForKids': 'True', 'BusinessAcceptsCredit...","Specialty Schools, Education, Gyms, Sports Clu...","{'Monday': '0:0-0:0', 'Tuesday': '12:0-22:0', ..."
623,CC7YefPM9zc080JHOKfI1w,Xtreme Adventures,24810 State Rd 54,Lutz,FL,33559,28.185208,-82.406265,2.5,26,0,"{'GoodForKids': 'True', 'BikeParking': 'True',...","Amusement Parks, Arts & Entertainment, Kids Ac...","{'Monday': '12:0-19:0', 'Wednesday': '17:0-21:..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149898,vQHLyvsUCUOpGOsPBDlP9g,Flex Appeal,"1101 Steamboat Pkwy, Ste 350",Reno,NV,89521,39.423649,-119.740235,4.5,37,1,"{'WiFi': 'u'free'', 'BikeParking': 'True', 'Bu...","Health & Medical, Food, Active Life, Chiroprac...","{'Monday': '0:0-0:0', 'Tuesday': '0:0-0:0', 'W..."
149992,aZz7sJReopPCV9KWvKSTng,Verticity Indoor Cycling,1008 Russell St,Nashville,TN,37206,36.175770,-86.751410,5.0,21,0,"{'BusinessAcceptsCreditCards': 'True', 'BikePa...","Cardio Classes, Trainers, Cycling Classes, Act...","{'Monday': '5:45-20:0', 'Tuesday': '5:45-20:0'..."
150072,9vP3RrHfPGqZW6V1QE0i0Q,Club Fitness - Creve Coeur,957 Woodcrest Executive Dr,Creve Coeur,MO,63141,38.674600,-90.451742,2.5,81,1,"{'GoodForKids': 'False', 'BusinessParking': '{...","Active Life, Tanning Beds, Tanning, Cardio Cla...","{'Monday': '0:0-0:0', 'Tuesday': '0:0-0:0', 'W..."
150334,LJ4GjQ1HL6kqvIPpNUNNaQ,Shanti Yoga and Ayurveda,"1638 Pine St, Fl 1",Philadelphia,PA,19103,39.945966,-75.169666,4.5,39,1,"{'ByAppointmentOnly': 'True', 'GoodForKids': '...","Health & Medical, Yoga, Shopping, Naturopathic...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ..."


In [33]:
doctor_df

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
128,P8brGDYVWjeW9GrKixH0yQ,Laser Spine Institute,5332 Avion Park Dr,Tampa,FL,33607,27.958613,-82.532595,3.0,36,0,"{'BusinessAcceptsCreditCards': 'True', 'ByAppo...","Medical Centers, Doctors, Diagnostic Services,...","{'Monday': '8:0-20:0', 'Tuesday': '8:0-20:0', ..."
137,E_kLUlU60fzoZYAOxbH_3w,Pain Institute of Southern Arizona,"4582 N 1st Ave, Ste 170",Tucson,AZ,85718,32.289183,-110.957230,2.5,43,1,"{'AcceptsInsurance': 'True', 'ByAppointmentOnl...","Pain Management, Doctors, Health & Medical, Os...","{'Monday': '7:0-16:30', 'Tuesday': '7:0-16:30'..."
156,t5tBd4p7BA7hNDitBUXxrw,Boise Therapeutic Massage Center,7153 W Emerald St,Boise,ID,83704,43.611949,-116.271110,4.0,68,1,"{'WheelchairAccessible': 'True', 'BusinessAcce...","Massage, Health & Medical, Medical Spas, Beaut...","{'Monday': '9:0-21:0', 'Tuesday': '9:0-21:0', ..."
281,6bFx8j2KqPfmT0XEUS1RMg,Center City Emergency Dentist,"1429 Walnut St, Fl 6",Philadelphia,PA,19102,39.949734,-75.165436,3.5,120,1,"{'BusinessAcceptsCreditCards': 'True', 'WiFi':...","Orthodontists, Health & Medical, Oral Surgeons...","{'Monday': '9:0-13:0', 'Tuesday': '9:0-18:0', ..."
356,yooufdsf3eAVq_9RQNSDJA,Mazzoni Center,"1348 Bainbridge St, Fl 2",Philadelphia,PA,19147,39.942627,-75.165083,3.0,46,1,"{'ByAppointmentOnly': 'True', 'AcceptsInsuranc...","Local Services, Community Service/Non-Profit, ...","{'Monday': '0:0-0:0', 'Tuesday': '9:0-19:0', '..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149957,KoE-tbMBF33yHhuALmIezw,Spavia Nashville,"8135 Sawyer Brown Rd, Ste 701",Nashville,TN,37221,36.081586,-86.947850,4.5,46,1,"{'WiFi': 'u'free'', 'WheelchairAccessible': 'T...","Reflexology, Massage Therapy, Skin Care, Beaut...","{'Monday': '0:0-0:0', 'Tuesday': '9:0-20:0', '..."
150076,GV9PSOhl1X-7pmRMbPif3g,IU Health North Hospital,11700 N Meridian St Ave,Carmel,IN,46032,39.959315,-86.159850,4.0,28,1,"{'AcceptsInsurance': 'True', 'BusinessAcceptsC...","Health & Medical, Hospitals","{'Monday': '8:0-20:0', 'Tuesday': '8:0-20:0', ..."
150173,E36h-bJ20aVz_b3-mQCoDQ,Massage & Bodywork By Kuryn,"4320 West El Prado Blvd, Ste 25",Tampa,FL,33629,27.911418,-82.517345,5.0,39,1,"{'BusinessAcceptsCreditCards': 'True', 'Busine...","Massage Therapy, Beauty & Spas, Massage, Healt...","{'Monday': '10:0-18:0', 'Tuesday': '10:0-18:0'..."
150318,VKbQHHUu_cB7M6jQwA3n-w,Planned Parenthood - FifthStreet Health Center,455 W 5th St,Reno,NV,89503,39.530298,-119.820694,3.5,20,1,"{'BusinessAcceptsCreditCards': 'True', 'ByAppo...","Obstetricians & Gynecologists, Health & Medica...","{'Monday': '0:0-0:0', 'Tuesday': '9:0-20:0', '..."


In [34]:
jewel_df

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
101,jiLGdnf42eYftkqfU2ze1w,Saks Fifth Avenue,8701 Keystone Crossing,Indianapolis,IN,46240,39.913836,-86.107907,3.5,29,1,"{'RestaurantsPriceRange2': '4', 'BusinessAccep...","Women's Clothing, Men's Clothing, Accessories,...","{'Monday': '10:0-18:0', 'Tuesday': '10:0-18:0'..."
600,VEbHYioBfoPiIOWntE_DBA,Tienda Ho,1105 State St,Santa Barbara,CA,93101,34.422317,-119.703667,4.5,34,0,"{'RestaurantsPriceRange2': '3', 'BusinessParki...","Shopping, Women's Clothing, Jewelry, Fashion, ...","{'Monday': '10:0-18:0', 'Tuesday': '10:0-18:0'..."
747,8f9Jqi4L69SQZIkU5r66Lw,Lee Read Diamonds,650 E Sonata Ln,Meridian,ID,83642,43.594074,-116.386033,3.5,20,1,"{'BusinessParking': '{'garage': False, 'street...","Jewelry Repair, Local Services, Shopping, Appr...","{'Monday': '0:0-0:0', 'Tuesday': '10:0-17:0', ..."
1001,PmASdGXX3XVokN2q92JIXg,L Priori Jewelry,"1601 Walnut St, Ste 1218",Philadelphia,PA,19102,39.949988,-75.167496,5.0,29,1,"{'BusinessAcceptsCreditCards': 'True', 'Wheelc...","Shopping, Jewelry","{'Monday': '9:0-17:0', 'Tuesday': '9:0-17:0', ..."
1246,w1A_N-pg-netgS_znGJlhg,Barn Attic,324 Harleysville Pike Rte113,Harleysville,PA,19438,40.280749,-75.393467,2.5,20,1,"{'RestaurantsPriceRange2': '3', 'BusinessAccep...","Fashion, Home & Garden, Used, Vintage & Consig...",None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149136,VvaRxOJbjaY9DbJ1FNFY6Q,POPPY - Santa Barbara,911 State St,Santa Barbara,CA,93101,34.420759,-119.701150,3.5,25,1,"{'BusinessAcceptsCreditCards': 'True', 'BikePa...","Accessories, Fashion, Shopping, Home Decor, Ho...","{'Monday': '11:0-17:0', 'Tuesday': '11:0-17:0'..."
149739,5gB1VjsYqzDeTnqWjX_7bQ,Harland Watch and Diamond,5043 S McCarran Blvd,Reno,NV,89502,39.478935,-119.784326,4.0,43,1,"{'BusinessAcceptsCreditCards': 'True', 'Busine...","Jewelry, Shopping, Local Services, Watches, Wa...","{'Tuesday': '10:0-17:0', 'Wednesday': '10:0-17..."
149866,7RoarRQWRh9V4JtiQUd_EA,Saks Fifth Avenue,1 Plaza Frontenac Street,St. Louis,MO,63131,38.631179,-90.407398,3.5,31,1,"{'RestaurantsPriceRange2': '3', 'BusinessAccep...","Accessories, Men's Clothing, Shopping, Departm...","{'Monday': '10:0-17:0', 'Tuesday': '12:0-17:0'..."
150106,eXT4nQTSqM79nwXhNw5UPg,Precious Metals Refinery,7069 E Tanque Verde Rd,Tucson,AZ,85715,32.252510,-110.841546,4.5,21,1,"{'BusinessParking': '{'garage': False, 'street...","Gold Buyers, Shopping, Jewelry, Pawn Shops, Wa...","{'Monday': '0:0-0:0', 'Tuesday': '10:0-18:0', ..."


In [35]:
hotel_df

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
55,xM6LoUcnpDpMBzXs_7dXAg,Fairfield Inn & Suites,719 E Baltimore Pike,Kennett Square,PA,19348,39.856248,-75.694610,3.0,37,1,"{'BusinessAcceptsCreditCards': 'True', 'WiFi':...","Hotels, Hotels & Travel, Event Planning & Serv...",None
65,uczmbBk5O3tYhGue13dCDg,New Orleans Spirit Tours,723 St Peter St,New Orleans,LA,70130,29.958431,-90.065173,4.0,38,1,{'WiFi': 'u'no''},"Hotels & Travel, Tours, Local Flavor","{'Monday': '0:0-0:0', 'Tuesday': '10:0-20:0', ..."
112,sB45WFgysT617bKWP_WJwA,Budweiser Brewery Experience,1200 Lynch St,Saint Louis,MO,63118,38.600197,-90.213538,4.5,605,1,"{'BikeParking': 'True', 'BusinessAcceptsCredit...","Bars, Beer Gardens, Food, Breweries, Nightlife...","{'Monday': '0:0-0:0', 'Tuesday': '9:0-17:0', '..."
155,hUQ9Z7kQeabvhPOAQOVV1A,Rathbone Mansions,1244 Esplanade Ave,New Orleans,LA,70116,29.967055,-90.065828,3.5,67,1,"{'WiFi': 'u'free'', 'BusinessAcceptsCreditCard...","Hotels, Hotels & Travel, Bed & Breakfast, Even...",None
181,ORL4JE6tz3rJxVqkdKfegA,Gaylord Opryland Resort & Convention Center,2800 Opryland Dr,Nashville,TN,37214,36.211592,-86.694319,3.0,1639,1,"{'BusinessAcceptsCreditCards': 'True', 'ByAppo...","Venues & Event Spaces, Performing Arts, Arts &...","{'Monday': '0:0-0:0', 'Tuesday': '0:0-0:0', 'W..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149925,NagkUmgLHFd1s8RTk1tcKw,Old Father Inn,4080 W Ina Rd,Marana,AZ,85653,32.338120,-111.053073,3.0,38,1,"{'Alcohol': 'u'full_bar'', 'RestaurantsPriceRa...","Karaoke, Steakhouses, Hotels, Hotels & Travel,...","{'Monday': '10:0-2:0', 'Tuesday': '10:0-2:0', ..."
149956,XGGPXLaa_B2Qc79cq9YPlg,Beach House Inn,320 W Yanonali St,Santa Barbara,CA,93101,34.410388,-119.695894,4.5,96,1,"{'RestaurantsPriceRange2': '2', 'BusinessAccep...","Apartments, Hotels, Hotels & Travel, Home Serv...",None
150035,sfGALNhZEYz4HyrVVRaf3A,Beach House Suites by the Don Cesar,3860 Gulf Blvd,St. Pete Beach,FL,33706,27.716203,-82.739108,3.5,39,1,"{'WiFi': ''free'', 'BusinessAcceptsCreditCards...","Hotels, Event Planning & Services, Hotels & Tr...","{'Monday': '0:0-0:0', 'Tuesday': '0:0-0:0', 'W..."
150046,1pBiQhcwaI_kF3urOdnG5A,Hotel Indigo Nashville,1719 W End Ave,Nashville,TN,37203,36.152989,-86.795709,3.0,39,0,"{'BusinessAcceptsCreditCards': 'True', 'Outdoo...","Hotels, Shopping, Hotels & Travel, Venues & Ev...",None


In [36]:
businessidlist = set(restaurant_df["business_id"])
businessidlist

{'kNUdugntWQD8jEB-QNFEjg',
 'Gx6VfCz3OTtcQYfsMucMGA',
 'tM-tPqhXx1ozGf_79dLJ_A',
 'xv_U__GI9gmzKIHT_TANnw',
 '0S3TvsmbCeOZ9UitnpRd1A',
 '11HXgzo47w5e5pziFXpTNg',
 'vUrTGX_7HxqeoQ_6QCVz6g',
 'KiIVFwppKrVY5ocNrPps1A',
 '8InvOBDq-jeVpsp-T8Wtng',
 'DGFmJcw2kmkLR4v4elw9NQ',
 'WftO0PbmpAAcRYvwjo0_Ug',
 'ZCURd9PiH4qdEjQfrIZzFg',
 'FR1SxQH_3ucer5lxyr2z1g',
 'uPQDMKhZWeTj2FAGdeNtYw',
 'Kf32m8URgzVGmaFCS9NPRQ',
 'LrkrvFHlnD1BHj2b2w0JpA',
 'EJIJ-QFdKRiPypfoPAVNBQ',
 '8C-G2bwZJLyThpUL5hzg9w',
 'N0z34U1SRJc3lTL2JUGa3w',
 'qiXPjUB2OgurOfSPjQ-P5Q',
 'UG9yclkxBcsHFb9U2Q_WdQ',
 'Km8zEoQ6KkkG0yVjiJXUCQ',
 'znrgu7fSTbFa9LVm__CV7A',
 'CS3dJ8m5rVd6u6bkqyt0lQ',
 'i9uiyuTQkePlDHMjD2a4Aw',
 'J938AdRE5sjveY7gdE9eMg',
 'HKweD7jQQpZgc6NaoltH2Q',
 'cEgsHkAnpYQbA60pIIZYKQ',
 'KkUyBysv7PbRXdEEZ4hNsw',
 'cSknwu7DA2QbbezL-hXbmQ',
 'T57pARC36apZzr3CSCs4-w',
 'lW5DyXf2pFQ_LFTacnGsCQ',
 'NJoYGf6P0qh3RSneQ0ARZQ',
 'Y5kZsua9U_GB7KPtRmaL0g',
 'AGZlpvY8u_WcmznFv0rlTA',
 'rbMtoYrV-R74EPfV-wjaRQ',
 'Ol4KW8Xx4lzPitSOw2ZdSA',
 

In [37]:
gymbusinessidlist = set(gym_df["business_id"])
gymbusinessidlist

{'-24TYSeA5yK8oH7hfJkZ0g',
 '-5uzjjKUA-kt81j605d5Jg',
 '-BvpRtFulI1rUkXEOXX3Tg',
 '-FVjMP72oLEDuiP-8gb2dA',
 '-JgPJ7mh3HAMwSEi6sHrRg',
 '-L6QwGtnjdW2VKnYHuu7lA',
 '-M7S_j79LiunqsLs1siEmQ',
 '-MHTa4ClIg83KKZpFARsSw',
 '-P1KJVrfiVncC3xka4GDoQ',
 '-RcCv4oCeDb49qnBGZgjgg',
 '-SL4pxguo0YQlEcOFZLEGQ',
 '-cDML6su_4Arm5USMTKNOw',
 '-cLmu1wRg6DXCTl_MDo8EA',
 '-cb3fjqcb9CVBi1bUYlyrA',
 '-kX4WpZWN_6Xp2mVsYoKVg',
 '-m2ucc0kb_S1-Y4J149AQw',
 '-r-yjid1guWGijXmAnsCNQ',
 '-rCB_kCW-u84S-CQir-lng',
 '-xV7ESuXJglzA_kdd8a7Mw',
 '-xu5l3Ptfwl2Dt5p6WTXvw',
 '-zGlHyBaxib9p7k18i5TSA',
 '04U01tL9wLxxUrmezl9BKw',
 '064WcuNRDGE_HDDt4BASpQ',
 '08Nop7lJ7oJuOQGGmKdfxg',
 '0A5BEEy08k79R5cVlRvJbA',
 '0LR3WhveQWj9zqCDFjshZA',
 '0MEx9QE4UwII2hkxOtc8CQ',
 '0Oyybjr6b142_t1GbmLLHA',
 '0XvVnDKixo4-BZwCN1Ehkg',
 '0bNbWkKmVxJmJuOZCtG7OQ',
 '0hgivhSXH9OCWRtlKHg6aA',
 '0jUNimaBy2JgkY65zu64lQ',
 '0kFnj6G-EQhRNCvHALK7uw',
 '0kWI_syvtjJMbCLhE75Cqw',
 '0pjv7o97ZoYPWuPRpfHDPw',
 '0pvEqchHi_fkxg2gbzt09A',
 '0y0qPSILVuwmj-hxkJJ0Bg',
 

In [38]:
docbusinessidlist = set(doctor_df["business_id"])
docbusinessidlist

{'luuBsXoVspG189QEnF7NKA',
 '82BEr5Vk1h4tIZAiA6IPGw',
 'Of8URWCdd2K3gyDp7DFsQg',
 '-T-TuWy5Kv72WEwwN-i2lg',
 'PpFeFOBwKBAYfYTVCcCquA',
 'UWxy45lXS_iZeptEaRnmYA',
 'EQjJzvWs72ZPzhLQC9SiPA',
 '9Axi4SWupbMO5peDQBYw1A',
 'IaqDpHScbazN5_I4Tt4Z3w',
 'X_vfIUXywYG0cjbd0H_TsQ',
 'PbeeSDun5zoTf1UDw6Cqpw',
 'Q0PURWx5cmU-VX4d-GNxOg',
 'sbzyPOmwTwcI-bagSDdx3Q',
 'b2cW30_3ZVMwe49VfaXPWA',
 'vUsNf2WAePvZT1nMccdUOA',
 'Uf_fLKjegPykEIsX-SnevQ',
 'VkTBf3l9vIeIMpga6vYTYQ',
 'kQl6RwRCicJmkxuUCBP1JA',
 'ARnUDTualdofPP0cpEkIxw',
 'PKAyJ93qVo8gDEkaDr5riA',
 'v_tEvT-3RRFHvP5hHfZYEg',
 'CaVQLG1eqn0zVu3gcpiqrQ',
 'UIvCmFje_AXFhSg34VlOiA',
 'I-SDICUJmdJev_Zwf-IKbw',
 '55GNarL5MuVzJEslW7QjbA',
 '_Hknj-SC2Z95iTTsOLpbOw',
 'Fb79FRiS7qUQIZfdTfx4eQ',
 'XszpGEsP_-UB7Da3WFd8jA',
 'zsBpKQiGTgMdfsRcyuSbgA',
 '96_c_7twb7hYRZ9HHrq01g',
 '0YoN2G_zfok7Sarx9d_1Wg',
 'v4-1uXmeVQH-G7RPZD1TUQ',
 '3pcnlpr00Ghz3N-oJobUYg',
 'FhII2Px_V80BAKN92KRf2A',
 'v3qTapJ-w2U5iycLNtXB6w',
 'pdc4NvS9dvOAH9O3N5INbQ',
 'rKm8EyKQRHxidls2Pns_cw',
 

In [39]:
jewelbusinessidlist = set(jewel_df["business_id"])
jewelbusinessidlist

{'-NfDXOlIoQtzA_V2Xhh4QQ',
 '-jdKfTfO0RiZxagql-Tcfg',
 '-r4K0ZM0cQ-cjLlH3S-LAA',
 '-v9IUORxPkzXAnSqP51rsA',
 '0-Tim4ucmy8bNugnzKXG8g',
 '0pzCUHzYxd69lgKfssaRyQ',
 '14TQ3FXCv8af1mGnRYEEYg',
 '1GSWARDKGoe2FWF1QeFqgA',
 '1Ksx9vG_RP5_TIxPONKqow',
 '1RqfozJoosHAsKZhc5PY7w',
 '1XxEqx1h0xpPLkqiqQn3QA',
 '1gT_8s71LAUkPjuj16sf-g',
 '1k5q8CJDRnyTPEkJyypMVQ',
 '1kRUUIg0EsHQndMfMC2x9g',
 '1l0nJA_oa_qY-AbwQSC8Uw',
 '1uLHetBaLyQheTfnf3t59g',
 '2IZwiQIIxbnjKGhUpbVlFg',
 '2OSMOlOmMVtBLoPN338vsQ',
 '2_K6jq54KRaLJaaVI8spKg',
 '38xDVi7BHRavXn6gCtlNvg',
 '39EG5mbSHIc8VaFb71Qaww',
 '3AuC8nsD7sUBihmcANx9Qw',
 '3SJKN_-whf5G40q183iTJg',
 '3Y4aOouZmRxYBhXP7h8wWw',
 '4H07EUUCT55Zt4TQubKmSA',
 '4R9OYjSy6EsKW-2_34HCTg',
 '4lwesYimP6HXbYGCL4cXdA',
 '50Q2uIgwUqmWGLjn_1HSlg',
 '56p17O_bHVEPBh4m41JAsA',
 '5A3MY4zE1e_L8d8yJZbsCg',
 '5gB1VjsYqzDeTnqWjX_7bQ',
 '5hHkrYVgGlUMDhz54H-YSw',
 '5hMuPN93RsEn-nmOfnC3FQ',
 '68KVeNtjmuDT4M0VDnVP9A',
 '6BKYnozWBMrXouTG8njQgQ',
 '6HDngyLgvaeRmftVmJuJzw',
 '6YPdxqjcNOxmu41Uf4x9RA',
 

In [40]:
hotelbusinessidlist = set(hotel_df["business_id"])
hotelbusinessidlist

{'auv_gvtR7fZp9b0mkEbTIA',
 'rqVTQHaxuivp8Z1_5_L6vw',
 'EF57en2jZd9e0m3E-TKV6Q',
 'k-2w4da77dGUjdyT0isVrA',
 'V8dZigOlI5fjpRxgOWwY3g',
 '8-B9Q7Wu3Z0RxYuz0HdlmA',
 'Oes3NdpCXGBA2w0BdXl0Ng',
 'NDwoKO79_T49UEKVDlHd3A',
 '7Cu3gElTkQEYIknEPsNaIw',
 'UtWJ6ULL3Pw9B9LXw4uSNQ',
 'YRYIopJGcNnUJ61TZXoZHQ',
 'Su8VY1rZQsOn9Wj53kuHdQ',
 'G9rBwcWp56C8r6ZFwVuimA',
 'jk8SnYWE1hHaFdFyCd897Q',
 'ngJLl0tdJPBgdUc-tn1Stg',
 'UVUq-80v1iW4pQiipttrow',
 'jMqEQYBIztLtcvnTujfHUA',
 'Zl5zVrIkTmGd8bP6dTJoyA',
 '7h-gAKxe_KqA5EDGCT4C2A',
 'yUxWRQMw74HrhhFJ3QsFnA',
 'zCwXtlag5ImXpYnrNrv5Gg',
 'JtwyVOjFQvI2VZwZHta-dw',
 '1KnmkIFa2gVwK4DavsqECA',
 'LnN94-sJc8U6shh_i0oclw',
 'Xo7fwCR-Qb_7zPdHdnzZgQ',
 'tnPaOSLr3cfpWcW7dSnDiQ',
 'rB2TBeaux5ClXtEZOTtqxQ',
 'iJ3A49gtUPspc7nd4Ov2HA',
 'JdzcnIxxvKnzsh01Zodt8w',
 'LD-SLkOwTUPt4mc3wIQ68A',
 'w4BzWRJ3xh0uRBvfqo28RA',
 'QQWQQaaHh7ZIx9pitLwnzg',
 'AAFofLiyt_K1Ar_eEuVc7g',
 'CeoYtlW6QIf2QdlmrB1fFQ',
 'RE5sfXu9N05KiAxv_2dR2A',
 '7MA_W6A7TKRustB9oTFpvQ',
 '-jlQofD5Qu3F4gpNHLsrOw',
 

In [1]:
from google.colab import drive
drive.mount('/content/drive')

#For Restaurants
file_path = "/content/drive/MyDrive/yelp_academic_dataset_review.json"
chunks = pd.read_json(file_path, lines=True, chunksize=100000)
# Save rows to a new df if the review is of a relevant business
# Create a new DF with the same columns as the reviews df
temp1_df = pd.DataFrame(columns=["business_id","name", "address", "city", "state", "postal_code", "latitude", "longitude", "stars", "review_count", "is_open", "attributes", "categories", "hours"])
filtered_rows = []
for chunk in chunks:
    filtered = chunk[chunk["business_id"].isin(businessidlist)]
    filtered_rows.append(filtered)


filtered_reviews_df = pd.concat(filtered_rows, ignore_index=True)
filtered_reviews_df
filtered_reviews_df.to_pickle("filteredreviewsdf.pkl")

ModuleNotFoundError: No module named 'google.colab'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

#For Gym
file_path = "/content/drive/MyDrive/yelp_academic_dataset_review.json"
chunks = pd.read_json(file_path, lines=True, chunksize=100000)
# Save rows to a new df if the review is of a relevant business
# Create a new DF with the same columns as the reviews df
temp2_df = pd.DataFrame(columns=["business_id","name", "address", "city", "state", "postal_code", "latitude", "longitude", "stars", "review_count", "is_open", "attributes", "categories", "hours"])
filtered_rows1 = []
for chunk in chunks:
    filtered1 = chunk[chunk["business_id"].isin(gymbusinessidlist)]
    filtered_rows1.append(filtered1)


# Save the new df using pickle, for fast access later
filtered_reviews_df1 = pd.concat(filtered_rows1, ignore_index=True)
filtered_reviews_df1
filtered_reviews_df1.to_pickle("filteredreviewsdf1.pkl")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

#For Doctors
file_path = "/content/drive/MyDrive/yelp_academic_dataset_review.json"
chunks = pd.read_json(file_path, lines=True, chunksize=100000)
# Save rows to a new df if the review is of a relevant business
# Create a new DF with the same columns as the reviews df
temp3_df = pd.DataFrame(columns=["business_id","name", "address", "city", "state", "postal_code", "latitude", "longitude", "stars", "review_count", "is_open", "attributes", "categories", "hours"])
filtered_rows2 = []
for chunk in chunks:
    filtered2 = chunk[chunk["business_id"].isin(docbusinessidlist)]
    filtered_rows2.append(filtered2)


# Save the new df using pickle, for fast access later
filtered_reviews_df2 = pd.concat(filtered_rows2, ignore_index=True)
filtered_reviews_df2
filtered_reviews_df2.to_pickle("filteredreviewsdf2.pkl")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

#For Jewelry Stores
file_path = "/content/drive/MyDrive/yelp_academic_dataset_review.json"
chunks = pd.read_json(file_path, lines=True, chunksize=100000)
# Save rows to a new df if the review is of a relevant business
# Create a new DF with the same columns as the reviews df
temp4_df = pd.DataFrame(columns=["business_id","name", "address", "city", "state", "postal_code", "latitude", "longitude", "stars", "review_count", "is_open", "attributes", "categories", "hours"])
filtered_rows3 = []
for chunk in chunks:
    filtered3 = chunk[chunk["business_id"].isin(jewelbusinessidlist)]
    filtered_rows3.append(filtered3)


# Save the new df using pickle, for fast access later
filtered_reviews_df3 = pd.concat(filtered_rows3, ignore_index=True)
filtered_reviews_df3
filtered_reviews_df3.to_pickle("filteredreviewsdf3.pkl")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

#For Hotels
file_path = "/content/drive/MyDrive/yelp_academic_dataset_review.json"
chunks = pd.read_json(file_path, lines=True, chunksize=100000)
# Save rows to a new df if the review is of a relevant business
# Create a new DF with the same columns as the reviews df
temp5_df = pd.DataFrame(columns=["business_id","name", "address", "city", "state", "postal_code", "latitude", "longitude", "stars", "review_count", "is_open", "attributes", "categories", "hours"])
filtered_rows4 = []
for chunk in chunks:
    filtered4 = chunk[chunk["business_id"].isin(hotelbusinessidlist)]
    filtered_rows4.append(filtered4)


# Save the new df using pickle, for fast access later
filtered_reviews_df4 = pd.concat(filtered_rows4, ignore_index=True)
filtered_reviews_df4
filtered_reviews_df4.to_pickle("filteredreviewsdf4.pkl")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
filtered_reviews_df = pd.read_pickle("filteredreviewsdf.pkl")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

#For Restaurants
file_path = "/content/drive/MyDrive/yelp_academic_dataset_review.json"
chunks = pd.read_json(file_path, lines=True, chunksize=100000)
# Save rows to a new df if the review is of a relevant business
# Create a new DF with the same columns as the reviews df
temp1_df = pd.DataFrame(columns=["business_id","name", "address", "city", "state", "postal_code", "latitude", "longitude", "stars", "review_count", "is_open", "attributes", "categories", "hours"])
filtered_rows = []
for chunk in chunks:
    filtered = chunk[chunk["business_id"].isin(businessidlist)]
    filtered_rows.append(filtered)


filtered_reviews_df = pd.concat(filtered_rows, ignore_index=True)
filtered_reviews_df
filtered_reviews_df.to_pickle("filteredreviewsdf.pkl")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
filtered_reviews_df1 = pd.read_pickle("filteredreviewsdf1.pkl")

In [ ]:
import pandas as pd
filtered_reviews_df2 = pd.read_pickle("filteredreviewsdf2.pkl")

In [ ]:
import pandas as pd
filtered_reviews_df3 = pd.read_pickle("filteredreviewsdf3.pkl")

In [ ]:
import pandas as pd
filtered_reviews_df4 = pd.read_pickle("filteredreviewsdf4.pkl")

In [ ]:
filtered_reviews_df

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,KU_O5udG6zpxOg-VcAEodg,mh_-eMZ6K5RLWhZyISBhwA,XQfwVwDr-v0ZS3_CbbE5Xw,3,0,0,0,"If you decide to eat here, just be aware it is...",2018-07-07 22:09:11
1,saUsX_uimxRlCVr67Z4Jig,8g_iMtfSiwikVnbP2etR0A,YjUWPpI6HXG530lwP-fb2A,3,0,0,0,Family diner. Had the buffet. Eclectic assortm...,2014-02-05 20:30:30
2,AqPFMleE6RsU23_auESxiA,_7bHUi9Uuf5__HHc_Q8guQ,kxX2SOes4o-D3ZQBkiMRfA,5,1,0,1,"Wow! Yummy, different, delicious. Our favo...",2015-01-04 00:01:03
3,Sx8TMOWLNuJBWer-0pcmoA,bcjbaE6dDog4jkNY91ncLQ,e4Vwtrqf-wpJfwesgvdgxQ,4,1,0,1,Cute interior and owner (?) gave us tour of up...,2017-01-14 20:54:15
4,JrIxlS1TzJ-iCu79ul40cQ,eUta8W_HdHMXPzLBBZhL1A,04UD14gamNjLY0IDYVhHJg,1,1,2,1,I am a long term frequent customer of this est...,2015-09-23 23:10:31
...,...,...,...,...,...,...,...,...,...
4522565,7NgXAuTFiJHYbuepOPwU0w,x1QLCwZGFAjxRRw4EHc3-g,1_BVWDzi5cVqWxNe9bOMMQ,5,1,0,1,Don't misinterpret my 5-star review....I don't...,2016-04-30 01:02:34
4522566,wD5ZWao_vjyT2h4xmGam8Q,7L7GL5Pi2cf8mbm2Dpw4zw,e_E-jq9mwm7wk75k7Yi-Xw,5,1,0,1,It is very rare for a restaurant to be this go...,2022-01-17 22:36:01
4522567,YVX1Wsa4LYxjvFwuHBb_gA,RKPkxOYQlM0BjhM-H6_vAw,X4mouE_cMiwbfyCPZ_K-FA,4,3,0,2,"Good, maybe very good. I went for lunch, so t...",2015-01-23 23:52:03
4522568,i-I4ZOhoX70Nw5H0FwrQUA,YwAMC-jvZ1fvEUum6QkEkw,Rr9kKArrMhSLVE9a53q-aA,5,1,0,0,For when I'm feeling like ignoring my calorie-...,2022-01-19 18:59:27


In [ ]:
filtered_reviews_df1

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,BiTunyQ73aT9WBnpR9DZGw,OyoGAe7OKpv6SyGZT5g77Q,7ATYjTIgM3jUlt4UM3IypQ,5,1,0,1,I've taken a lot of spin classes over the year...,2012-01-03 15:28:18
1,r-PjI5sBvNoBekk5mURNww,uzy_KYIZx65cp7Yh8_seeQ,ZuM1vcZ_ObCjCCGweYzItg,5,15,0,2,"I took the beginning class there, and I loved ...",2013-10-24 16:40:47
2,-pBDXBop_8v1dKk-BBpyzQ,YW0CUT7vA7pqn2f6aL8rlg,vPlhhf6CcB5dzQv1sEwlvg,5,1,1,1,Planet fitness gets four stars just because it...,2014-09-25 11:44:08
3,LF5UQBZuqjU5B4kTVSIKGg,FsvBdHsES26G0LB2MQZcUg,_agzW_0F9uSJ7i6EJ9ufeg,5,1,0,1,"Rebecca didn't Doula my birth. BUT, she was o...",2015-01-04 01:42:59
4,QS3gJVX4WAfk0Chd4W3zIw,HzX3oBnPUEImHhsdxCu0Aw,ZGD4WacjO1v4Ath3Qil-Sg,5,2,0,1,We belong to lifetime in another state and wer...,2015-10-23 13:44:28
...,...,...,...,...,...,...,...,...,...
39653,zQXCf-ig90eRFkIfSYC2uw,Q2cW75ZwfVKPpOrVwAtMYg,9vP3RrHfPGqZW6V1QE0i0Q,1,0,0,0,BEWARE when you sign up to look at the process...,2015-11-02 17:29:14
39654,1_ulO0tLglmyYQfuXjPQUg,sRayUe6Y_YSXOK7pKfdlBg,IzLOX3tUTqQFBb9kHdu_wA,3,9,0,2,"Tons of top-roping, lead routes, and space... ...",2011-01-27 16:49:38
39655,mq_C9IbtFrquPLwLl5PwhQ,x5YqLqPA5h_G8lmat5BGzA,sEunc4jrrrXsRcWrDjsa-A,1,2,0,0,Beware- don't come here if you want to actuall...,2019-11-15 19:48:36
39656,E5sXn_qdPX5qRxUEApmiuA,Mc4C7fVY0sEcD-U5eOA2Og,ZNfph3_VzRLTOemk9Tmzaw,3,5,0,1,This gym gets the job done. \n\nUpsides: \nChi...,2016-01-25 21:45:34


In [ ]:
filtered_reviews_df2

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,rj0asGlfKjlXH6iEgu-jdw,wA7V8Ijgbv-16rmnwW-hjA,LfYUOclwLs_SPrNAdKMjsw,5,4,0,0,Nice relaxing place to get a massage! Same day...,2013-11-14 04:02:52
1,LF5UQBZuqjU5B4kTVSIKGg,FsvBdHsES26G0LB2MQZcUg,_agzW_0F9uSJ7i6EJ9ufeg,5,1,0,1,"Rebecca didn't Doula my birth. BUT, she was o...",2015-01-04 01:42:59
2,dQ3EU6cevDqHAr_ygy1O8A,CNyXcn0c0V5CFmigqqw-Xg,oY5LFo6Yxxf32ePna6mEUQ,5,1,0,0,"I absolutely love this place!\n\nGreat hours, ...",2014-12-30 17:55:51
3,eBrVkzHhHKZxyVS1SPKbJw,1PJ-RjMqXHeymSkLRzuIdA,nOyphAl0JQ8JrvNi93pclQ,2,3,1,1,So I no longer go here which is a shame becaus...,2014-10-28 13:58:00
4,ydXJhVuaE6r7kN1niSqA1w,iyocBf1XVHvsr3SZMpfSnA,BMMcP9fBunmTyYY6XdtQtw,1,0,0,0,LYING SCAMMERS. BAIT AND SWITCH. Will try and ...,2018-01-10 01:35:47
...,...,...,...,...,...,...,...,...,...
107305,3CPM4NFv23mEA5kgnBzNdg,E2w1VFGN44k_bMHnuLsnUQ,H7DVitiflHl8D9mkRxDGoQ,4,0,0,0,"Been struggling with a terrible tooth, hoping ...",2015-05-13 19:42:47
107306,Q6h57S-h4-ND2slRKUg1wg,VjiqLNphmE322B7rDuNoSQ,e3BJXpNS5pjcdXycdxagJw,1,0,0,0,I only went here for Dr Shah for about 8 years...,2020-03-10 16:58:16
107307,7zN73BqyXsuzW8NuyNHkvQ,WLlNXb9kWLhdEbmmWkZeFQ,k9qFNafEUmNbCb1bky5swA,2,1,0,0,UPDATE: Clayton Med Spa reached out to me 6 we...,2019-09-18 02:20:00
107308,os2N5i_n6qlGSeoNIKXEig,hZSWEjV0JAAQ9nq0k50aNg,jz_2B9gw8Ho18ZJmi-iyRw,4,0,0,0,You can always count on a great massage here! ...,2022-01-08 00:54:17


In [ ]:
filtered_reviews_df3

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,hONoNzvKbDWlB2AlAF-6vw,gj2a3Pdr85S476EDcVtr2A,50Q2uIgwUqmWGLjn_1HSlg,5,0,0,0,Replaced a dive watch battery in less than 5 m...,2015-06-08 18:04:32
1,bIFYvAi3D5HdSvOjTI63GQ,A95oGBHdDNsE6vHwiwLufA,Af2UlNzQSiZCapJ6qtwu6g,4,0,0,0,Great store if you like 50's-Midcentury modern...,2012-06-26 13:22:19
2,HrYtx4z77i84rgXgOJ60HA,mcx0aeOgvt4Yul5lrvLPeg,u_S84xiDCRJ3CV204CIYuQ,4,0,0,2,I love going into this place to just browse an...,2018-03-16 02:53:19
3,TAArVUfx2VH90L7eXzK1uA,4mjnkd8oJVCfBKN3i4rB-g,u_S84xiDCRJ3CV204CIYuQ,4,0,0,0,"This is my kind of store! A unique, quirky, a...",2010-08-21 04:26:21
4,KZkwG-a3Dm1hJCQ3JX7dXg,NcxVQhNlw219Grdp0yEP8Q,50Q2uIgwUqmWGLjn_1HSlg,5,1,0,0,My grandmother has been going to this place fo...,2014-06-28 05:39:56
...,...,...,...,...,...,...,...,...,...
17224,NgUrGPW6DMjSQMACEbbvlA,ZWQyRKsDXKfYrP5hmlJvfA,eXT4nQTSqM79nwXhNw5UPg,4,2,0,2,Redeem. \n\nWebster says:\n\n1a : to buy back ...,2021-09-10 15:23:20
17225,sibo0WoPJeEBkZ_6ebBRlQ,1fMCwimeMSVq-XfcmIjIiQ,Gv1viz1bePCzeWDNc4tosA,2,0,1,0,Such mixed feelings about this place. One of m...,2021-08-01 01:31:56
17226,Fab-eUPLMp8-sozD1fribQ,EA-bfqoCUGzvaHUQ35pbMg,7RoarRQWRh9V4JtiQUd_EA,1,9,0,0,It has now been a week since my horrific exper...,2019-04-13 20:44:53
17227,sskFUA-ZkKE123kCOkkzOw,rX25Ai-2nC18LOBStyPs_w,EEtATHOs8ZY6evvbrf4PtA,1,13,0,0,"So like many others on here, I'm a fan of Kris...",2019-07-12 22:13:49


In [ ]:
filtered_reviews_df4

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,6AxgBCNX_PNTOxmbRSwcKQ,r3zeYsv1XFBRA4dJpL78cw,gmjsEdUsKpj9Xxu6pdjH0g,5,0,2,0,Loved this tour! I grabbed a groupon and the p...,2015-01-03 23:21:18
1,pHwbdway4yeI-dSSmZA7-Q,qEEk0PuoH1dVa619t8fgpw,PY9GRfzr4nTZeINf346QOw,4,0,0,0,We checked in around 2:30 pm. Check-in was qu...,2017-09-20 16:16:47
2,ra9fNjYUumKp_iaqv-jjDg,l_izt6hn7oTxalU06mSyqg,jTI5Xjk27An8ceJ6VwpXiQ,4,0,0,0,"Comfortable bed, good breakfast, fast internet...",2013-08-15 15:57:09
3,qdzNocGBnh8U-cvE_N8qbA,Fq7d14GDAsGjDoojP_ygOQ,pAgtmlIGqFYaWdBoCUeitw,1,3,0,0,The TV shows are $4.99 and they have commercia...,2012-12-04 02:40:49
4,HuWEnZr7-0HveaqXxyywgg,Eb8LgZArKmW0KgW6lTni-w,YbnJYHNp_fHbI-hcFg48vQ,5,2,0,2,The kayaking tour at the Santa Cruz Island was...,2009-08-03 21:55:27
...,...,...,...,...,...,...,...,...,...
300684,s6xifrpyKVupSUgzTj0M2w,uPwlsNBDEGvWriW1pjt1ww,suFQTjOdHk2LM-t0jV98jA,1,0,0,0,I think this is the worst hotel I have been to...,2019-08-07 21:05:26
300685,dnGhGSnMlhZSqVtWeCpagg,eWybZmo5BZl3cp6v4_L7rg,BON1JqGZyG2D1m1Uc9AP9g,1,0,0,0,I've been a customer for over 30 years. Unfor...,2017-02-22 14:18:53
300686,Y7b6Z6sCZiWFWHz8ARba-g,4dtO8AYcpN1T0oqfyoqqiQ,mkXJKCb8bbaiqzY1MGzosw,2,1,0,0,Have enjoyed this place tremendously the past ...,2016-09-05 00:43:28
300687,_b_SOIkWHsNPRGW4RM3X5w,L_fdAnrH5Pxi4PqF08_ToA,ez4kMLP6OJEIaMbMrrGRdA,5,1,1,1,My sister and I went on the ghost tour with Th...,2018-12-10 16:31:32


Combine/merge the review and business data???

In [ ]:
# Research how to prepare the tables for NN analysis.
# We have reviews with business id.
# We have another table that links the business id to to star-counts.
# Merge these tables such that the NN system can create a formula that predicts stars
# based on the text reviews.

# Likely involve adding a column to the reviews table that is the number of stars that
# the business has.

# Left join of reviews / businesses on business_id column (I think!)

Split into training and test

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import layers

Baseline Model

In [ ]:
#Import relevent packages
from tensorflow import keras
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from collections.abc import Sequence
from sklearn import preprocessing
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
import shutil
import os


Implement TF-IDF

In [ ]:
#combine reviews per business so each business has one large string of their reviews
df_review_per_business = filtered_reviews_df.groupby('business_id')['text'].apply(' '.join).reset_index()
df_review_per_business = df_review_per_business.rename(columns={'text': 'all_text_reviews'})

#merge with business_df to bring in the star rating
merge = pd.merge(df_review_per_business, business_df[['business_id','stars']], on = 'business_id')

#apply TF-IDF on all text reviews
vectorizer = TfidfVectorizer(max_features = 5, min_df = 2, max_df =0.8)
X = vectorizer.fit_transform(merge['all_text_reviews'])

In [ ]:
y = merge['stars'] #labels AKA business star ratings

Build and Train the Baseline Model

In [ ]:
# Build the Model Skeleton (Input layer, 1-2 hidden layers and output layer)
model = keras.Sequential( [
    keras.Input(shape=(X.shape[1],)),
    keras.layers.Dense(128, activation = 'relu'),
    keras.layers.Dense(1)
])

In [ ]:
#Train/Test Split
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

Compile the Model

In [ ]:
#Compile Model MSE good for regression,
model.compile(loss = 'mean_squared_error', optimizer = 'adam', metrics = [tf.keras.metrics.RootMeanSquaredError()])

Add Callbacks

In [ ]:
#Setup earlyStopping callback with patience so training wont overfill
monitor = EarlyStopping(monitor = 'val_loss', min_delta = 1e-3, patience = 5, verbose = 1, mode = 'auto')

#save the  model to baselinemodel
checkpointer = ModelCheckpoint(filepath = 'baseline_model.keras', verbose = 1, save_best_only = True)

Train/Fit Model

In [ ]:
#utilizing the x_train, y_train here reserve 20% of the data for validation
model.fit(x_train, y_train, validation_split = 0.2, callbacks = [monitor, checkpointer], verbose = 2, epochs = 1000)

Epoch 1/1000

Epoch 1: val_loss improved from inf to 0.57288, saving model to baseline_model.keras
637/637 - 4s - 6ms/step - loss: 1.6376 - root_mean_squared_error: 1.2797 - val_loss: 0.5729 - val_root_mean_squared_error: 0.7569
Epoch 2/1000

Epoch 2: val_loss improved from 0.57288 to 0.55337, saving model to baseline_model.keras
637/637 - 2s - 3ms/step - loss: 0.5342 - root_mean_squared_error: 0.7309 - val_loss: 0.5534 - val_root_mean_squared_error: 0.7439
Epoch 3/1000

Epoch 3: val_loss did not improve from 0.55337
637/637 - 2s - 3ms/step - loss: 0.5288 - root_mean_squared_error: 0.7272 - val_loss: 0.5568 - val_root_mean_squared_error: 0.7462
Epoch 4/1000

Epoch 4: val_loss improved from 0.55337 to 0.54948, saving model to baseline_model.keras
637/637 - 2s - 3ms/step - loss: 0.5284 - root_mean_squared_error: 0.7269 - val_loss: 0.5495 - val_root_mean_squared_error: 0.7413
Epoch 5/1000

Epoch 5: val_loss did not improve from 0.54948
637/637 - 3s - 4ms/step - loss: 0.5268 - root_mean_sq

In [ ]:
#Evaluate model
model.evaluate(x_test, y_test)

266/266 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.5077 - root_mean_squared_error: 0.7123


[0.5213911533355713, 0.7220742106437683]

In [ ]:
#save the vectorizer
import pickle
with open('vectorizer.pkl', 'wb') as f:
    pickle.dump(vectorizer, f)

In [ ]:
#use the baseline model to make predictions
predictions= model.predict(x_test).flatten()


266/266 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step


In [ ]:
#predictions DataFrame setup
ids = merge['business_id'].values

x_train, x_test, y_train, y_test, ids_train, ids_test = train_test_split(X, y, ids, test_size = 0.25, random_state = 42)

In [ ]:
#Build the Dataframe
results = pd.DataFrame({'business_id': ids_test, 'correct_stars': y_test, 'predicted_stars': predictions })

In [ ]:
#save to CSV
results.to_csv('results.csv', index=False)

In [ ]:
results.head() #test to see if the results outputed correctly

,business_id,correct_stars,predicted_stars
23234,forbtWAsfRrLbumFA7cCZA,4.0,3.713623
10887,JJlePhVjtWL_CieAcdqCRg,3.5,3.650236
7209,CMovGmSWFCjhOBOYiymetg,4.0,3.540065
25585,kAmXP3e66dxu21-5_9sBsw,4.0,3.627665
13438,ODDbQCvJBFwLDimkT9KoYA,4.0,3.669206


In [ ]:
!ls -lh #confirm file is in directory


total 3.4G
-rw-r--r-- 1 root root  32K Sep 26 05:20 baseline_model.keras
-rw-r--r-- 1 root root  52M Sep 26 04:19 businessdf.pkl
drwx------ 5 root root 4.0K Sep 26 03:40 drive
-rw-r--r-- 1 root root  32M Sep 26 04:27 filteredreviewsdf1.pkl
-rw-r--r-- 1 root root  82M Sep 26 04:30 filteredreviewsdf2.pkl
-rw-r--r-- 1 root root  13M Sep 26 04:32 filteredreviewsdf3.pkl
-rw-r--r-- 1 root root 239M Sep 26 04:36 filteredreviewsdf4.pkl
-rw-r--r-- 1 root root 2.9G Sep 26 04:41 filteredreviewsdf.pkl
-rw-r--r-- 1 root root 305K Sep 26 06:05 results.csv
drwxr-xr-x 1 root root 4.0K Sep 24 13:41 sample_data
-rw-r--r-- 1 root root  949 Sep 26 05:42 vectorizer.pkl
-rw-r--r-- 1 root root 114M Sep 26 03:45 yelp_academic_dataset_business.json


**HYPERPARAMETER SWEEPS**

In [ ]:
#Results Table
results = pd.DataFrame(columns=["Model", "Activation", "Optimizer", "Layers", "Test_RMSE"])


In [ ]:
#Build Model 2
model = keras.Sequential( [
    keras.Input(shape=(X.shape[1],)),
    keras.layers.Dense(128, activation = 'sigmoid'),
    keras.layers.Dense(1)
])

In [ ]:
#Compile the model
model.compile(loss = 'mean_squared_error', optimizer = 'adam', metrics = [tf.keras.metrics.RootMeanSquaredError()])

In [ ]:
#Fit
model.fit(x_train, y_train, validation_split = 0.2, callbacks = [monitor, checkpointer], verbose = 2, epochs = 1000)

Epoch 1/1000

Epoch 1: val_loss did not improve from 0.53879
637/637 - 3s - 5ms/step - loss: 1.3568 - root_mean_squared_error: 1.1648 - val_loss: 0.5633 - val_root_mean_squared_error: 0.7505
Epoch 2/1000

Epoch 2: val_loss did not improve from 0.53879
637/637 - 2s - 3ms/step - loss: 0.5335 - root_mean_squared_error: 0.7304 - val_loss: 0.5549 - val_root_mean_squared_error: 0.7449
Epoch 3/1000

Epoch 3: val_loss did not improve from 0.53879
637/637 - 2s - 4ms/step - loss: 0.5326 - root_mean_squared_error: 0.7298 - val_loss: 0.5590 - val_root_mean_squared_error: 0.7477
Epoch 4/1000

Epoch 4: val_loss did not improve from 0.53879
637/637 - 2s - 3ms/step - loss: 0.5331 - root_mean_squared_error: 0.7301 - val_loss: 0.5555 - val_root_mean_squared_error: 0.7453
Epoch 5/1000

Epoch 5: val_loss did not improve from 0.53879
637/637 - 2s - 2ms/step - loss: 0.5346 - root_mean_squared_error: 0.7312 - val_loss: 0.5531 - val_root_mean_squared_error: 0.7437
Epoch 6/1000

Epoch 6: val_loss did not impro

In [ ]:
#Evaluate model
model.evaluate(x_test, y_test)

266/266 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.5194 - root_mean_squared_error: 0.7205


[0.5339400172233582, 0.7307119965553284]

In [ ]:
#Save model2
checkpointer = ModelCheckpoint(filepath="model_sigmoid_adam.keras", save_best_only=True)

In [ ]:
#Append results
loss, rmse = model.evaluate(x_test, y_test, verbose=0)

In [ ]:
#Add Baseline behavior
results = pd.concat([results,pd.DataFrame([{"Model": "Model 1","Activation": "relu","Optimizer": "adam","Layers": "[128]","Test_RMSE": 0.7213
    }])
], ignore_index=True)


/tmp/ipython-input-522045455.py:2: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results = pd.concat([results,pd.DataFrame([{"Model": "Model 1","Activation": "relu","Optimizer": "adam","Layers": "[128]","Test_RMSE": 0.7213


In [ ]:
results = pd.concat([results, pd.DataFrame([{"Model": "Model 2", "Activation": "sigmoid", "Optimizer": "adam", "Layers": "[128]", "Test_RMSE": 0.7205}]) ], ignore_index=True)

In [ ]:
print(results)

     Model Activation Optimizer Layers  Test_RMSE
0  Model 1       Relu      Adam    128     0.7213
1  Model 2    Sigmoid      Adam  [128]     0.7205


In [ ]:
#Build Model3
model = keras.Sequential( [
    keras.Input(shape=(X.shape[1],)),
    keras.layers.Dense(64, activation = 'tanh'),
    keras.layers.Dense(1)
])

In [ ]:
#Compile the model
model.compile(loss = 'mean_squared_error', optimizer = 'adam', metrics = [tf.keras.metrics.RootMeanSquaredError()])

In [ ]:
#Fit
model.fit(x_train, y_train, validation_split = 0.2, callbacks = [monitor, checkpointer], verbose = 2, epochs = 1000)

Epoch 1/1000
637/637 - 3s - 4ms/step - loss: 1.5276 - root_mean_squared_error: 1.2360 - val_loss: 0.6614 - val_root_mean_squared_error: 0.8133
Epoch 2/1000
637/637 - 2s - 3ms/step - loss: 0.5808 - root_mean_squared_error: 0.7621 - val_loss: 0.5721 - val_root_mean_squared_error: 0.7564
Epoch 3/1000
637/637 - 3s - 4ms/step - loss: 0.5353 - root_mean_squared_error: 0.7317 - val_loss: 0.5561 - val_root_mean_squared_error: 0.7457
Epoch 4/1000
637/637 - 2s - 2ms/step - loss: 0.5342 - root_mean_squared_error: 0.7309 - val_loss: 0.5662 - val_root_mean_squared_error: 0.7524
Epoch 5/1000
637/637 - 2s - 2ms/step - loss: 0.5341 - root_mean_squared_error: 0.7308 - val_loss: 0.5657 - val_root_mean_squared_error: 0.7521
Epoch 6/1000
637/637 - 2s - 2ms/step - loss: 0.5334 - root_mean_squared_error: 0.7303 - val_loss: 0.5516 - val_root_mean_squared_error: 0.7427
Epoch 7/1000
637/637 - 2s - 2ms/step - loss: 0.5341 - root_mean_squared_error: 0.7308 - val_loss: 0.5547 - val_root_mean_squared_error: 0.7448

In [ ]:
#Evaluate model
model.evaluate(x_test, y_test)

266/266 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.5182 - root_mean_squared_error: 0.7197


[0.5327320098876953, 0.729884922504425]

In [ ]:
#Save model3
checkpointer = ModelCheckpoint(filepath="model_tanh_adam.keras", save_best_only=True)

In [ ]:
#Append results
loss, rmse = model.evaluate(x_test, y_test, verbose=0)

In [ ]:
results = pd.concat([results, pd.DataFrame({"Model": "Model 3", "Activation": "tanh", "Optimizer": "adam", "Layers": "[64]", "Test_RMSE": 0.7197}, index=[0])], ignore_index=True)

In [ ]:
#Build Model4
model = keras.Sequential( [
    keras.Input(shape=(X.shape[1],)),
    keras.layers.Dense(128, activation = 'relu'),
    keras.layers.Dense(1)
])

In [ ]:
#Compile the model
model.compile(loss = 'mean_squared_error', optimizer = 'sgd', metrics = [tf.keras.metrics.RootMeanSquaredError()])

In [ ]:
#Fit
model.fit(x_train, y_train, validation_split = 0.2, callbacks = [monitor, checkpointer], verbose = 2, epochs = 1000)

Epoch 1/1000
637/637 - 2s - 3ms/step - loss: 0.7215 - root_mean_squared_error: 0.8494 - val_loss: 0.5626 - val_root_mean_squared_error: 0.7501
Epoch 2/1000
637/637 - 1s - 2ms/step - loss: 0.5352 - root_mean_squared_error: 0.7316 - val_loss: 0.5582 - val_root_mean_squared_error: 0.7471
Epoch 3/1000
637/637 - 2s - 4ms/step - loss: 0.5298 - root_mean_squared_error: 0.7279 - val_loss: 0.5501 - val_root_mean_squared_error: 0.7417
Epoch 4/1000
637/637 - 2s - 3ms/step - loss: 0.5285 - root_mean_squared_error: 0.7269 - val_loss: 0.5510 - val_root_mean_squared_error: 0.7423
Epoch 5/1000
637/637 - 1s - 2ms/step - loss: 0.5277 - root_mean_squared_error: 0.7264 - val_loss: 0.5514 - val_root_mean_squared_error: 0.7425
Epoch 6/1000
637/637 - 1s - 2ms/step - loss: 0.5275 - root_mean_squared_error: 0.7263 - val_loss: 0.5535 - val_root_mean_squared_error: 0.7440
Epoch 7/1000
637/637 - 1s - 2ms/step - loss: 0.5269 - root_mean_squared_error: 0.7258 - val_loss: 0.5513 - val_root_mean_squared_error: 0.7425

In [ ]:
#Evaluate model
model.evaluate(x_test, y_test)

266/266 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 13.5087 - root_mean_squared_error: 3.6754


[13.527921676635742, 3.678032398223877]

In [ ]:
#Save model4
checkpointer = ModelCheckpoint(filepath="model_relu_sgd.keras", save_best_only=True)

In [ ]:
#Append results
loss, rmse = model.evaluate(x_test, y_test, verbose=0)

In [ ]:
results = pd.concat([results, pd.DataFrame({"Model": "Model 4", "Activation": "relu", "Optimizer": "sgd", "Layers": "[128]", "Test_RMSE": 0.7219}, index=[0])], ignore_index=True)

In [ ]:
results.to_csv("hyperparam_results.csv", index=False)

In [ ]:
results.head()

,Model,Activation,Optimizer,Layers,Test_RMSE
0,Model 1,relu,adam,[128],0.7213
1,Model 2,sigmoid,adam,[128],0.7205
2,Model 3,tanh,adam,[64],0.7197
3,Model 4,relu,sgd,[128],0.7219


Based on the hyperparam data chart, Model 3 is the best model due to the RMSE score being the lowest, indicating that it predicted Yelp Ratings a little more accurate then others

Collect Model Data

In [ ]:
# Collect data from model
predictions = model.predict(x_test).flatten()
actuals = y_test.values

Load Relevant Business Data For Evaluation

In [ ]:
"""
Load business data and predictions for evaluation
        
Parameters:
-----------
business_data : pd.DataFrame
    DataFrame containing business information (must have 'business_name' column)
predictions : np.array
    Model predictions for business ratings
actuals : np.array
    Actual business ratings
"""
business_data = business_df.copy()
predictions_data = np.array(predictions)
actuals_data = np.array(actuals)

n_predictions = len(predictions_data)
business_data = business_df.head(n_predictions).copy()
        
# Combine data for easier manipulation
business_data['predicted_rating'] = predictions_data
business_data['actual_rating'] = actuals_data

Calculate Root Mean Square Error (RMSE)

In [ ]:
"""
Calculate Root Mean Square Error
        
Parameters:
-----------
predictions : np.array, optional
    Model predictions (uses instance data if None)
actuals : np.array, optional
    Actual values (uses instance data if None)
            
Returns:
--------
float : RMSE value
"""
preds = predictions if predictions is not None else predictions_data
acts = actuals if actuals is not None else actuals_data
        
if preds is None or acts is None:
    raise ValueError("No predictions or actuals provided")
            
rmse = np.sqrt(mean_squared_error(acts, preds))

print(f"RMSE Test: {rmse:.4f}")

Random Business Test

In [ ]:
"""
Randomly select businesses and display their predictions vs actuals
        
Parameters:
-----------
n_businesses : int
    Number of businesses to showcase
seed : int, optional
    Random seed for reproducibility
            
Returns:
--------
pd.DataFrame : Formatted table of selected businesses
"""
n_businesses = 5
seed = None

if business_data is None:
    raise ValueError("No business data loaded. Call load_data() first.")
            
if seed is not None:
    random.seed(seed)
    np.random.seed(seed)
        
valid_data = business_data.dropna(subset=['predicted_rating', 'actual_rating'])
    
if len(valid_data) == 0:
    raise ValueError("No valid business data with both predictions and actuals found.")
    
# Randomly select businesses
sample_businesses = business_data.sample(n=min(n_businesses, len(business_data)))
        
# Create formatted display table
showcase_data = []
for idx, business in sample_businesses.iterrows():
    business_name = str(business['name'])
    if len(business_name) > 35:
        business_name = business_name[:32] + "..."
            
    business_info = {
        'Business Name': business_name,
        'City': business['city'],
        'State': business['state'],
        'Actual Rating': f"{business['actual_rating']:.2f}",
        'Predicted Rating': f"{business['predicted_rating']:.2f}",
        'Difference': f"{business['predicted_rating'] - business['actual_rating']:+.2f}",
        'Reviews': int(business['review_count']) if pd.notna(business['review_count']) else 0
    }
    showcase_data.append(business_info)
        
showcase_df = pd.DataFrame(showcase_data)
print(showcase_df.to_string(index=False))

Full Model Evaulation

In [ ]:
"""
Comprehensive model evaluation
        
Parameters:
-----------
detailed_output : bool
    Whether to print detailed results
            
Returns:
--------
Dict : Dictionary containing evaluation metrics
"""
if predictions_data is None or actuals_data is None:
    raise ValueError("No data loaded. Call load_data() first.")
        
# Calculate basic metrics
preds = predictions if predictions is not None else predictions_data
acts = actuals if actuals is not None else actuals_data
        
if preds is None or acts is None:
    raise ValueError("No predictions or actuals provided")
            
rmse = np.sqrt(mean_squared_error(acts, preds))
print(f"RMSE Test: {rmse:.4f}")

detailed_output = True

mae = np.mean(np.abs(predictions_data - actuals_data))
correlation = np.corrcoef(predictions_data, actuals_data)[0, 1]
        
# Calculate prediction statistics
pred_stats = {
    'mean': np.mean(predictions_data),
    'std': np.std(predictions_data),
    'min': np.min(predictions_data),
    'max': np.max(predictions_data)
}
        
actual_stats = {
    'mean': np.mean(actuals_data),
    'std': np.std(actuals_data),
    'min': np.min(actuals_data),
    'max': np.max(actuals_data)
}
        
results = {
    'rmse': rmse,
    'mae': mae,
    'correlation': correlation,
    'prediction_stats': pred_stats,
    'actual_stats': actual_stats,
    'n_samples': len(predictions_data)
}
        
if detailed_output:
    print("="*50)
    print("MODEL EVALUATION RESULTS")
    print("="*50)
    print(f"Number of samples: {results['n_samples']}")
    print(f"RMSE: {rmse:.4f}")
    print(f"MAE: {mae:.4f}")
    print(f"Correlation: {correlation:.4f}")
    print("\nPrediction Statistics:")
    print(f"  Mean: {pred_stats['mean']:.4f}")
    print(f"  Std:  {pred_stats['std']:.4f}")
    print(f"  Range: [{pred_stats['min']:.4f}, {pred_stats['max']:.4f}]")
    print("\nActual Statistics:")
    print(f"  Mean: {actual_stats['mean']:.4f}")
    print(f"  Std:  {actual_stats['std']:.4f}")
    print(f"  Range: [{actual_stats['min']:.4f}, {actual_stats['max']:.4f}]")
    print("="*50)

Generate and Test Lift Chart

In [ ]:
"""
Create a lift chart showing model performance across prediction deciles
        
Parameters:
-----------
n_bins : int
    Number of bins/deciles for the lift chart (default: 10)
save_path : str, optional
    Path to save the plot
show_plot : bool
    Whether to display the plot
"""

n_bins = 10
save_path = None
show_plot = True

if predictions_data is None or actuals_data is None:
    raise ValueError("No data loaded. Call load_data() first.")

# Create a temporary dataframe for binning
df_temp = pd.DataFrame({
    'predicted': predictions_data,
    'actual': actuals_data
})

# Sort by predicted values and create bins
df_temp = df_temp.sort_values('predicted', ascending=False)
df_temp['bin'] = pd.qcut(df_temp['predicted'], q=n_bins, labels=False, duplicates='drop')

# Calculate lift metrics for each bin
lift_data = []
for bin_num in sorted(df_temp['bin'].unique()):
    bin_data = df_temp[df_temp['bin'] == bin_num]
    
    lift_metrics = {
        'bin': bin_num + 1,  # Start from 1 instead of 0
        'count': len(bin_data),
        'avg_predicted': bin_data['predicted'].mean(),
        'avg_actual': bin_data['actual'].mean(),
        'min_predicted': bin_data['predicted'].min(),
        'max_predicted': bin_data['predicted'].max()
    }
    lift_data.append(lift_metrics)

lift_df = pd.DataFrame(lift_data)

# Create the lift chart
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Plot 1: Average Predicted vs Actual by Decile
x_pos = lift_df['bin']
ax1.bar(x_pos - 0.2, lift_df['avg_predicted'], width=0.4, 
        label='Avg Predicted', alpha=0.8, color='skyblue')
ax1.bar(x_pos + 0.2, lift_df['avg_actual'], width=0.4, 
        label='Avg Actual', alpha=0.8, color='lightcoral')

ax1.set_xlabel('Prediction Decile (1=Highest Predicted)')
ax1.set_ylabel('Average Rating')
ax1.set_title('Lift Chart: Predicted vs Actual Ratings by Decile')
ax1.legend()
ax1.grid(True, alpha=0.3)
ax1.set_xticks(x_pos)

# Plot 2: Lift (Actual/Overall Average)
overall_avg = df_temp['actual'].mean()
lift_df['lift'] = lift_df['avg_actual'] / overall_avg

bars = ax2.bar(x_pos, lift_df['lift'], alpha=0.8, color='green')
ax2.axhline(y=1.0, color='red', linestyle='--', label='Baseline (1.0)')
ax2.set_xlabel('Prediction Decile (1=Highest Predicted)')
ax2.set_ylabel('Lift (Actual/Overall Average)')
ax2.set_title('Lift Chart: Performance Lift by Decile')
ax2.legend()
ax2.grid(True, alpha=0.3)
ax2.set_xticks(x_pos)

# Add value labels on bars
for i, bar in enumerate(bars):
    height = bar.get_height()
    ax2.text(bar.get_x() + bar.get_width()/2., height,
            f'{height:.2f}', ha='center', va='bottom')

plt.tight_layout()

if save_path:
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    
if show_plot:
    plt.show()
else:
    plt.close()


In [ ]:
print("\n" + "="*50)
print("ALL TESTS COMPLETED SUCCESSFULLY!")
print("="*50)